In [2]:
import logging

_logger = logging.getLogger("retraction_experiments")
logging.basicConfig(
    filename="retraction_experiments.log",
    format="%(asctime)s %(levelname)s:%(name)s:%(message)s",
    level=logging.DEBUG,
    force=True,
)

## TODO

* Lots of ev just has PA IDs -- should convert back to PMIDs?

In [3]:
from wags_tails import CustomData
from wags_tails.utils.downloads import handle_zip, download_http
from wags_tails.utils.versioning import DATE_VERSION_PATTERN
from datetime import datetime, timezone

def _latest_version_cb() -> str:
    return datetime.now(tz=timezone.utc).strftime(DATE_VERSION_PATTERN)

def _download_cb(version, outfile_path) -> None:
        tqdm_params = {
            "disable": False,
            "unit": "B",
            "ncols": 80,
            "unit_divisor": 1024,
            "unit_scale": True,
        }
        download_http(
            "https://api.pharmgkb.org/v1/download/file/data/occurrences.zip",
            outfile_path,
            handler=handle_zip,
            tqdm_params=tqdm_params,
        )

data_fetcher = CustomData(
    "pharmgkb",
    "tsv",
    _latest_version_cb,
    _download_cb,
)

pharmgkb_data = data_fetcher.get_latest()

In [4]:
from retraction_audit.audit import RetractionLookup
import os

os.environ["RETRACTION_WATCH_EMAIL"] = "james.sharpsteen@gmail.com"
os.environ["NCBI_AUTH_EMAIL"] = "james.sharpsteen@gmail.com"

lookup = RetractionLookup(True)

In [9]:
import polars as pl
from pathlib import Path
from tqdm import tqdm
from retraction_audit.citation_tools import SqlitePmidCache, get_pmid_from_pmcid

df = pl.read_csv(pharmgkb_data[0], separator="\t")
os.environ["NCBI_AUTH_EMAIL"] = "james.sharpsteen@gmail.com"

pmid_cache = SqlitePmidCache(Path("pmid_cache.db"))
pmids = set()

for source_id in tqdm(df["Source ID"]):
    if source_id.startswith("PMID"):
        pmid = source_id[5:]
    elif source_id.startswith("PMC"):
        try:
            pmid = get_pmid_from_pmcid(source_id, pmid_cache)
        except KeyError:
            _logger.info("Unable to find PMID for PMCID %s", source_id)
            continue
    else:
        _logger.info("Unrecognized form of identifier: %s", source_id)
    pmids.add(pmid)

100%|█████████████████████████████████████████████████████████████████████████████████| 119301/119301 [00:41<00:00, 2863.01it/s]


In [31]:
retracted_ev = []
for pmid in pmids:
    retraction = lookup.get_retraction_by_pmid(pmid)
    if retraction:
        retracted_ev.append(retraction)
print(len(retracted_ev))

2


In [1]:
retracted_ev

NameError: name 'retracted_ev' is not defined

In [18]:
from tools import store_results
store_results("PharmGKB", len(retracted_ev), retracted_ev, len(pmids))